# Imports

In [1]:
import numpy as np
import pandas as pd
import os, json, math, time

In [2]:
!pip install yelpapi
!pip install tqdm

In [2]:
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [3]:
#load API Credentials
with open('/Users/17175/.secret/yelp_api.json','r') as f:
    login = json.load(f)

In [4]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [6]:
#instantiate yelpapi variable
yelp = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp

# Define search terms and file paths

In [7]:
location = 'Lancaster, PA 17603'
term = 'ramen'

In [8]:
location.split(',')[0]

'Lancaster'

In [19]:
#specify folder for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)
#specifying json_file filename
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [20]:
JSON_FILE

'Data/Lancaster-ramen.json'

In [22]:
#check if file exists
file_exists = os.path.isfile(JSON_FILE)
#if it does not exist
if file_exists == False:
    #Create folder
    #get folder name
    folder = os.path.dirname(JSON_FILE)

    #if jsonfile included a folder:
    if len(folder)>0:
        #create folder
        os.makedirs(folder,exist_ok = True)

    #inform user and save empty list
    print(f"[i] (JSON_FILE) not found. Saving empty list to file.")

    #save first page of results
    with open(JSON_FILE, 'w') as f:
        json.dump((), f)

#If it exists, inform user
else:
    print(f"[i] (JSON_FILE) already exists.")

[i] (JSON_FILE) not found. Saving empty list to file.


# determine how many results there are and figure how many pages of results is needed

In [23]:
#load previous results (there shouldn't be any)
with open(JSON_FILE, 'r') as f:
    previous_results= json.load(f)

#set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [25]:
#perform first call
results = yelp.search_query(location=location, term=term, offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [26]:
#how many results total?
total_results = results['total']
total_results

14

In [27]:
#how many did we get the details for?
results_per_page =len(results['businesses'])
results_per_page

14

In [28]:
#join new results with old list with extend and save to file
previous_results.extend(results['businesses'])
with open(JSON_FILE, 'w') as f:
    json.dump(previous_results,f)

# Convert .json to dataframe

In [29]:
final_df= pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,n4GL78xZDW2FJomsjITjGg,ninja-ramen-lancaster,Ninja Ramen,https://s3-media1.fl.yelpcdn.com/bphoto/Iw3Lgz...,False,https://www.yelp.com/biz/ninja-ramen-lancaster...,26,"[{'alias': 'ramen', 'title': 'Ramen'}]",4.5,"{'latitude': 40.074193163754, 'longitude': -76...",[],"{'address1': '2046 Fruitville Pike', 'address2...",17178859036,(717) 885-9036,8589.063222,NaN
1,QIuOfv6Sipw_gyjJZ690iQ,issei-noodle-lancaster-lancaster,Issei Noodle Lancaster,https://s3-media4.fl.yelpcdn.com/bphoto/WFxud1...,False,https://www.yelp.com/biz/issei-noodle-lancaste...,294,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.0,"{'latitude': 40.039017, 'longitude': -76.306519}",[delivery],"{'address1': '44 N Queen St', 'address2': 'Ste...",17174496800,(717) 449-6800,6654.382822,$$
2,FKDrzrfmx5YuVkBZ-JHX4w,noodle-shack-lititz-2,Noodle Shack,https://s3-media3.fl.yelpcdn.com/bphoto/BFl_l1...,False,https://www.yelp.com/biz/noodle-shack-lititz-2...,75,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.0,"{'latitude': 40.130175, 'longitude': -76.307694}","[pickup, delivery]","{'address1': '1032 Lititz Pike', 'address2': '...",17176270375,(717) 627-0375,14913.010138,NaN
3,iUx7FF6UM3TF9evk-JbVsQ,onions-cafe-lancaster,Onion's Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/RXNuBo...,False,https://www.yelp.com/biz/onions-cafe-lancaster...,76,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"{'latitude': 40.043698, 'longitude': -76.3069658}","[pickup, delivery]","{'address1': '340 N Queen St', 'address2': '',...",17173968777,(717) 396-8777,6934.240186,$
4,nFei3TlPFDLgIKJ0iqzEvQ,shogun-japanese-steakhouse-and-sushi-lancaster-2,Shogun Japanese Steakhouse and Sushi,https://s3-media1.fl.yelpcdn.com/bphoto/HFgUbT...,False,https://www.yelp.com/biz/shogun-japanese-steak...,81,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 40.066578, 'longitude': -76.337663}","[pickup, delivery]","{'address1': '680 Park City Ctr', 'address2': ...",17172991000,(717) 299-1000,7409.197307,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
9,8dmbYfe0BlqN9Jmp0P8xKQ,spring-house-brewing-company-lancaster,Spring House Brewing Company,https://s3-media2.fl.yelpcdn.com/bphoto/31WmwL...,False,https://www.yelp.com/biz/spring-house-brewing-...,214,"[{'alias': 'breweries', 'title': 'Breweries'}]",3.5,"{'latitude': 40.03022, 'longitude': -76.30838}",[delivery],"{'address1': '209 Hazel St', 'address2': '', '...",17179842530,(717) 984-2530,6001.292310,$$
10,7JBQ0PB_bSfawA-qf_xPEw,hunger-n-thirst-lancaster,Hunger N Thirst,https://s3-media1.fl.yelpcdn.com/bphoto/ALsTon...,False,https://www.yelp.com/biz/hunger-n-thirst-lanca...,197,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 40.0531058, 'longitude': -76.3255...",[delivery],"{'address1': '920 Landis Ave', 'address2': '',...",17172083808,(717) 208-3808,6597.005586,$$
11,oHUQ7sB8_NsuQwud7x5shA,root-lancaster,Root,https://s3-media3.fl.yelpcdn.com/bphoto/scOnXZ...,False,https://www.yelp.com/biz/root-lancaster?adjust...,216,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",4.0,"{'latitude': 40.0425848856228, 'longitude': -7...",[delivery],"{'address1': '223 W Walnut St', 'address2': ''...",17175170918,(717) 517-0918,6648.707622,$$
12,5xKDaxm9lLfUBwg286adZQ,nissin-foods-usa-lancaster,Nissin Foods USA,,False,https://www.yelp.com/biz/nissin-foods-usa-lanc...,1,"[{'alias': 'ramen', 'title': 'Ramen'}]",1.0,"{'latitude': 40.05222, 'longitude': -76.38885}",[],"{'address1': '2901 Hempland Rd', 'address2': '...",17172850945,(717) 285-0945,5459.203578,NaN
13,pj97qeiwdyZgQaCqXsGXZg,eastern-palace-lancaster,Eastern Palace,https://s3-media3.fl.yelpcdn.com/bphoto/NL7KWz...,False,https://www.yelp.com/biz/eastern-palace-lancas...,152,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 40.0394087363819, 'longitude': -7...","[pickup, delivery]","{'address1': '2206 Columbia Ave', 'address2': ...",17172086356,(717) 208-6356,3860.130608,$$
